# 基于Mindspore实现二分类损失函数

通过实验了解分类损失函数原理，并能够基于Mindspore框架实现分类损失函数的计算以及训练和预测。

## 1、实验目的
* 学会内置损失函数的使用。
* 掌握Mindspore中多种损失函数的特点和应用场景。


## 2、二分类损失函数原理介绍
损失函数（loss function）或也称为代价函数（cost function），亦称目标函数，是将随机事件或其有关随机变量的取值映射为非负实数以表示该随机事件的“风险”或“损失”的函数，用于衡量预测值与真实值差异的程度。在实际的机器学习应用中，损失函数通常会作为学习准则与优化问题相联系，通过最小化损失函数求解和评估模型。

(1) $n n$. L1Loss:计算预测值和目标值之间的平均绝对误差:
$$
\ell(x, y)=L=\left\{l_1, \ldots, l_N\right\}^{\top}, \quad \text { with } l_n=\left|x_n-y_n\right|
$$
其中N为数据集中的 batch_size 值。
$$
\ell(x, y)= \begin{cases}\operatorname{mean}(L), & \text { if reduction }=\text { 'mean'; } \\ \operatorname{sum}(L), & \text { if reduction='sum' }\end{cases}
$$
nn.L1Loss 中的参数 reduction 取值可为 mean， sum，或 none 。若 reduction 为 mean 或 sum，则输出一个经过均值或求和后的标量 Tensor (降维) ；若 reduction 为 none，则所输出Tensor的shape为广播后的shape。

(2)平均绝对误差MAE(Mean Absolute Error):计算模型预测值 f(x) 与样本真实值 y 之间距离的平均值。
$$\mathrm{MAE}=\frac{1}{m} \sum_{i=1}^{m}\left|y_{i}-f\left(x_{i}\right)\right|$$

## 3、实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.4；Python环境=3.11


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理

### 4.1数据准备

   在本次实验中，我们使用numpy从均匀分布中随机生成测试数据x，从正态分布中随机生成噪声加入因变量y。

In [1]:
import numpy as np                         # Python 的科学计算库，用于处理矩阵和数组等数据结构。
def get_data(num, w=2.0, b=3.0):           # 生成数据及对应标签   
    for _ in range(num):                   #  num=160,生成160个样本点
        x = np.random.uniform(-10.0, 10.0) # 生成服从(-10.0, 10.0)范围内的均匀分布的元素，返回值的元素类型为浮点型。
        noise = np.random.normal(0, 1)     # 随机产生一个服从正态分布(0,1)的数值
        y = x * w + b + noise              # 增加噪音生成y
        yield np.array([x]).astype(np.float32), np.array([y]).astype(np.float32)#将数组元素类型转换为float32位;
                                           # 我们为了提高效率，并不一次性返回所有数据，而是采用迭代器形式返回单一数据。
#可视化部分生成数据
eval_data=list(get_data(5))
#zip()函数迭代eval_data，将eval_data中的元素打包成一个个元组，然后返回由这些元组组成的列表。
x,y=zip(*eval_data)
#可视化生成的5个样本点
eval_data 

[(array([6.374636], dtype=float32), array([16.205915], dtype=float32)),
 (array([-5.7773037], dtype=float32), array([-9.919436], dtype=float32)),
 (array([-0.23789468], dtype=float32), array([3.2614572], dtype=float32)),
 (array([-8.137355], dtype=float32), array([-12.005216], dtype=float32)),
 (array([8.518956], dtype=float32), array([20.397629], dtype=float32))]

### 4.2数据加载
使用GeneratorDataset，通过迭代列表构造数据集，指定生成数据集的列名为data和lable，使用batch函数指定每个批处理数据包含的数据条目。

In [2]:
from mindspore import dataset as ds
def create_dataset(num_data, batch_size=16):                           #加载数据集
    data=list(get_data(num_data))
    dataset = ds.GeneratorDataset(data, column_names=['data', 'label'])#指定生成数据集的列名为data和lable
    dataset = dataset.batch(batch_size)                                #设置数据批次
    return dataset        

## 5、几个损失函数

### 5.1内置损失函数
下面介绍 mindspore. $\mathrm{nn}$ 模块中内置的损失函数L1损失。

In [3]:
# 1.内置损失函数
import mindspore as ms
import mindspore.nn as nn 
loss = nn.L1Loss()                       # 输出loss均值
loss_sum = nn.L1Loss(reduction='sum')    # 输出loss和
loss_none = nn.L1Loss(reduction='none')  # 输出loss原值
input_data = ms.Tensor(np.array([1, 0, 1, 0, 1, 0]).astype(np.float32)) # 定义输入数据
target_data = ms.Tensor(np.array([0, 0, 1, 1, 1, 0]).astype(np.float32)) # 定义标签
print("loss:", loss(input_data, target_data))             # 打印loss均值
print("loss_sum:", loss_sum(input_data, target_data))     # 打印所有loss和
print("loss_none:\n", loss_none(input_data, target_data)) # 打印每个样本点loss的原值

loss: 0.33333334
loss_sum: 2.0
loss_none:
 [1. 0. 0. 1. 0. 0.]


### 5.2基于nn.Cell构造损失函数

nn.Cell 是MindSpore的基类，不但可用于构建网络，还可用于定义损失函数。使用 $n n$.Cell定义损失函数的过程与定义一个普通的网络相似，差别在于，其执行逻辑部分要计算的是前向网络输出与真实值之间的误差。

In [4]:
# 2.基于nn.Cell构造损失函数
import mindspore.ops as ops
class MAELoss(nn.Cell):                 # 自定义损失函数MAELoss
    def __init__(self):                 # 初始化
        super(MAELoss, self).__init__()
        self.abs = ops.abs
        self.reduce_mean = ops.ReduceMean()
    def construct(self, base, target):  # 调用算子        
        x = self.abs(base - target)
        return self.reduce_mean(x)
loss = MAELoss()                        # 定义损失函数
input_data = ms.Tensor(np.array([1, 0, 1, 0, 1, 0]).astype(np.float32))  # 定义输入数据
target_data = ms.Tensor(np.array([0, 0, 1, 1, 1, 0]).astype(np.float32)) # 定义标签
output = loss(input_data, target_data)  # 计算损失
print(output)                           # 打印损失

0.33333334


### 5.3基于nn.LossBase构造损失函数

基于nn.LossBase构造损失函数MAELoss与基于nn.Cell构造损失函数的过程类似，都要重写__init__方法和construct方法。

nn.LossBase可使用方法get_loss将reduction应用于损失计算。

In [5]:
# 3.基于nn.LossBase构造损失函数
class MAELoss(nn.LossBase):               # 自定义损失函数MAELoss
    def __init__(self, reduction="mean"): # 初始化并求loss均值       
        super(MAELoss, self).__init__(reduction)
        self.abs = ops.abs              # 求绝对值算子
    def construct(self, base, target):    # 调用算子
        x = self.abs(base - target)
        return self.get_loss(x)           # 返回loss均值
loss = MAELoss()                          # 定义损失函数
input_data = ms.Tensor(np.array([1, 0, 1, 0, 1, 0]).astype(np.float32))  # 生成预测值
target_data = ms.Tensor(np.array([0, 0, 1, 1, 1, 0]).astype(np.float32))  # 生成真实值
output = loss(input_data, target_data)    # 计算损失
print(output)                             # 打印损失

0.33333334


## 6、模型构建

本次实验使用MindSpore中内置的L1loss损失函数和接口Model中fit接口进行模型训练，构造Model时需传入前向网络、损失函数和优化器，Model会在内部将它们关联起来，生成一个可用于训练的网络模型。指定一个回调函数LossMonitor来监控训练过程中的loss值，并将训练集大小传递给它,LossMonitor函数计算并输出每个epoch中的平均训练损失和损失。

In [6]:
#1.定义模型
#使用了MindSpore的神经网络模块中的dense函数，该函数用于创建全连接。这里创建了一个输入维度为1，输出维度为1的全连接层。
net = nn.Dense(1, 1)

#2.定义损失函数
#使用了MindSpore的神经网络模块中的L1Loss函数，该函数用于计算 L1 Loss（也称为绝对值误差）。
loss_fn = nn.L1Loss()

#3.定义优化器
#使用了MindSpore的优化器模块中的Momentum函数，它是一个带动量随机梯度下降法（SGD）。
#Momentum算法的基本思路是使用上一步的梯度方向来决定本步的梯度方向，从而解决随机梯度下降法中的震荡问题。
#它可以加速收敛，而且对于参数的自适应能力也更强。
#learning_rate为学习率，momentum为动量参数。
optimizer = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)

## 7、模型训练
生成num_data个数据点，实例化线性网络，选择L1Loss和Momentum优化器进行训练。

In [7]:
from mindspore.train import Model, MAE, LossMonitor 

#将定义好的神经网络模型、损失函数和优化器用Model函数封装，指定了评价指标为MAE（平均绝对误差）。
#这个函数在训练过程中会自动计算每个batch的损失值和评价指标，并使用优化器更新模型参数。
model = Model(net, loss_fn, optimizer, metrics={"MAE": MAE()})
train_dataset = create_dataset(num_data=160)          # 生成训练集
eval_dataset = create_dataset(num_data=160)           # 生成测试集
train_dataset_size = train_dataset.get_dataset_size() # 训练集大小

# 定义forward函数
def forward_fn(inputs, targets):
    logits = net(inputs)
    loss = loss_fn(logits, targets)
    return loss

grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters)

def train_step(inputs, targets):
    loss, grads = grad_fn(inputs, targets)
    optimizer(grads)
    return loss


## 8、模型预测
使用封装好的模型对训练集进行训练，并在验证集上进行评估训练。

In [ ]:
#指定了一个回调函数LossMonitor来监控训练过程中的loss值，并将训练集大小传递给它。回调函数可以在特定的阶段中被调用，如个epoch结束时。
#此处LossMonitor函数计算并输出每个epoch中的平均训练损失和损失。

from mindspore import ops, nn

# 定义验证函数
def eval_step(model, data, label):
    output = model.predict(data)
    metric = model._metrics['MAE']
    metric.update(output, label)
    return metric.eval()

# 主训练循环
for epoch in range(10):
    # 训练阶段
    for batch, (data, label) in enumerate(train_dataset):
        loss = train_step(data, label)
        if batch % 10 == 0:
            print(f"Epoch: {epoch}, Step: {batch}, Loss: {loss}")

    # 验证阶段

    model._metrics['MAE'].clear()  # 清空指标缓存
    for data, label in eval_dataset:
        eval_result = eval_step(model, data, label)
    print(f"Epoch: {epoch}, MAE: {eval_result}")





Epoch: 0, Step: 0, Loss: 10.05474
Epoch: 0, MAE: 5.344454717636109
Epoch: 1, Step: 0, Loss: 5.2119384
Epoch: 1, MAE: 3.40930290222168
Epoch: 2, Step: 0, Loss: 2.7828224
Epoch: 2, MAE: 3.2873810052871706
Epoch: 3, Step: 0, Loss: 3.3883517
Epoch: 3, MAE: 2.771801400184631
Epoch: 4, Step: 0, Loss: 2.847849
Epoch: 4, MAE: 2.3322375535964968
Epoch: 5, Step: 0, Loss: 2.190043
Epoch: 5, MAE: 1.8599430322647095
Epoch: 6, Step: 0, Loss: 2.0208054
Epoch: 6, MAE: 1.4688191175460816
Epoch: 7, Step: 0, Loss: 1.6750548
Epoch: 7, MAE: 1.1897496640682221
Epoch: 8, Step: 0, Loss: 1.3291664
Epoch: 8, MAE: 0.9495349526405334
Epoch: 9, Step: 0, Loss: 1.082624
Epoch: 9, MAE: 0.8861383855342865
